In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from datetime import datetime
import re
import swifter

In [2]:
DATA_DIR = "data"

In [3]:
#Extracting intericite data
times = pd.read_csv(DATA_DIR + "/export-intercites-gtfs-last/stop_times.txt")
stops = pd.read_csv(DATA_DIR + "/export-intercites-gtfs-last/stops.txt")
stops = stops[pd.notnull(stops["stop_name"])]
#Extracting ter data
stops_ter = pd.read_csv(DATA_DIR + "/ter/stops.txt")
stops_ter = stops_ter[pd.notnull(stops_ter["stop_name"])]
stop_times_ter = pd.read_csv(DATA_DIR + "/ter/stop_times.txt")
# Stations
stations = pd.read_csv('./data/_stations.csv')[['Name']]

In [4]:
#fix times which hours exceeds 24
def fix_times(time) : 
    
    if int(time.split(":")[0]) >= 24:
        return "{}:".format(int(time.split(":")[0]) - 24)+":".join(time.split(":")[1:])
    return time

def compute_duration(arrival_list, departure_list) : 
    durations = []
    for i in range(len(arrival_list) -1) : 
        durations.append((datetime.strptime(arrival_list[i+1],"%H:%M:%S") - datetime.strptime(departure_list[i],"%H:%M:%S")).seconds / 60)
    return durations

In [5]:
# fixing time for intercite
times["arrival_time"] = times["arrival_time"].apply(fix_times)
times["departure_time"] = times["departure_time"].apply(fix_times)
#fix time for ter
stop_times_ter["arrival_time"] = stop_times_ter["arrival_time"].apply(fix_times)
stop_times_ter["departure_time"] = stop_times_ter["departure_time"].apply(fix_times)

In [6]:
cols_1 = times.columns
cols_2 = stops.columns 

In [7]:
# Transformations to apply for station names.
NORMALIZATIONS = {'ç': 'c',
                  'é': 'e',
                  'è': 'e',
                  'ë': 'e',
                  'ô': 'o',
                  'â': 'a',
                  'î': 'i',
                  'ê': 'e',
                  '\\(': '',
                  '\\)': '',
                  'û': 'u',
                  '-': ' '}
NORMALIZATIONS_REGEX = {re.compile(k): v for k,v in NORMALIZATIONS.items()}
NORMALIZATIONS['('] = NORMALIZATIONS['\\(']
NORMALIZATIONS[')'] = NORMALIZATIONS['\\)']
del NORMALIZATIONS['\\(']
del NORMALIZATIONS['\\)']
WHITESPACE_REGEX = re.compile(' +')
PARAN_REGEX = re.compile(' *\([^)]*\)$')
# Some stations are spelled quite differently, or are ambiguous (had to resort to googling to figure out which is
# which exactly). We have to define these manually.
REPLACEMENTS = {'AEROPORT CDG 2 TGV ROISSY': 'Roissy-Aéroport-Charles-de-Gaulle 2-TGV (TGV)',
                'AEROPORT CDG2 TGV ROISSY': 'Roissy-Aéroport-Charles-de-Gaulle 2-TGV (TGV)',
                'Bâle': 'Bâle-St-Jean',
                'MASSY PALAISEAU': 'Massy-Palaiseau-Grande-Ceinture',
                'LILLE EUROPE-147322': 'Lille-Europe',
                'RUFFEC CHARENTE': 'Ruffec',
                'MOUTIERS SALINS BRIDES': 'Moûtiers-Salins-Brides-les-Bains',
                'MANTES LA J. TGV': 'Mantes-la-Jolie',
                'CLUSES  74': 'Cluses',
                'STRASBOURG': 'Strasbourg-Ville',
                'SAUMUR': 'Saumur-Rive-Droite',
                'PARIS MONTPARNAS VAUGIRARD BLS': 'Paris-Montparnasse',
                'PARIS MONTPARNASSE 3 VAUGIRARD': 'Paris-Montparnasse',
                'PLOUARET TREGOR': 'Plouaret',
                'plouaret tregor': 'Plouaret',
                'Saint Malo': 'St-Malo',
                'CONFLANS FDO TGV': 'Conflans-Fin-d\'Oise',
                'MULHOUSE VILLE': 'Mulhouse-Ville',
                'MULHOUSE': 'Mulhouse-Ville',
                'SAINT MICHEL VALLOIRE': 'St-Michel-Valloire',
                'JUVISY TGV': 'Juvisy',
                'Facture': 'Facture-Biganos',
                'Saint Maixent': 'St-Maixent (Deux-Sèvres)',
                'St maixent': 'St-Maixent (Deux-Sèvres)',
                'Vendôme Gare TGV': 'Vendôme',
                'saint nazaire': 'St-Nazaire',
                'Saint Brieuc': 'St-Brieuc',
                'PARIS MONTPARNASSE 1 ET 2': 'Paris-Montparnasse',
                'Dol de Bretagne': 'Dol',
                'DOL DE BRETAGNE': 'Dol',
                'LYON SAINT EXUPERY TGV': 'Lyon-St-Exupéry-TGV',
                'MONTPELLIER': 'Montpellier (CNM)',
                'AVIGNON SUD': 'Avignon-Centre',
                'Angers': 'Angers-St-Laud',
                'VILLENEUVE LES AVIGNONS': 'Villeneuve-lès-Avignon'}
# Some common suffixes which are missing in many cases, so we try using them to get a match when all else fails.
SUFFIXES = ['-tgv', '-voyageurs']
MEM = {}
UNKNOWNS = set()

def _from_name_get_matches(search, pattern):
    pattern = pattern.strip()
    if pattern in REPLACEMENTS:
        pattern = REPLACEMENTS[pattern]
    pattern = pattern.lower()
    for pat, repl in NORMALIZATIONS.items():
        pattern = pattern.replace(pat, repl)
    pattern = re.sub(WHITESPACE_REGEX, ' ', pattern)
    pattern = pattern.strip()
    return stations[search == pattern]

def _from_name_complex(name, try_suffixes=False):
    search = stations.Name.str.lower()
    for pat, repl in NORMALIZATIONS_REGEX.items():
        search = search.replace(pat, repl)
    pattern = name
    if try_suffixes:
        orig_pattern = pattern
        for suffix in SUFFIXES:
            pattern = orig_pattern + suffix
            match = _from_name_get_matches(search, pattern)
            if not match.empty:
                return match
    else:
        return _from_name_get_matches(search, pattern)

TRIMMINGS = ['-Centre', '-Carrefour', '-Mairie', 'Aéroport', '-Ctre', '-Gare-?.*', r' *\([^)]*\)']
TRIMMINGS = [re.compile(trimming + '$') for trimming in TRIMMINGS]
def _from_name(name, rec=False):
    if isinstance(name, list):
        return [_from_name(n) for n in name]
    elif isinstance(name, str):
        name = name.replace('Gare de ', '')
    else:
        return None
    match = _from_name_complex(name)
    if not match.empty:
        return match.Name.iloc[0]
    else:
        match = _from_name_complex(name, try_suffixes=True)
        if match is not None and not match.empty:
            return match.Name.iloc[0]
        elif name.endswith('-') or name.endswith('.'):
            return _from_name(name[:-1])
        elif rec:
            return None
        else:
            for trimming in TRIMMINGS:
                name = re.sub(trimming, '', name)
            return _from_name(name, rec=True)

def fix_names(names):
    # print('.', end='')
    result = []
    for name in names:
        if name in MEM:
            fixed_name = MEM[name]
        else:
            fixed_name = _from_name(name)
            MEM[name] = fixed_name
        if not fixed_name:
            UNKNOWNS.add(name)
        result.append(fixed_name)
    return result

In [8]:
table = pd.merge(times, stops, how = "left", left_on = times.stop_id, right_on= stops.stop_id).rename(columns = {"key_0" : "stop_id"})[[cols_1[0] , cols_1[1], cols_1[2] , cols_2[0],cols_2[1],cols_2[3],cols_2[4]]]
timetable = table.groupby("trip_id").agg(list).reset_index()

timetable["durations"] = timetable[["arrival_time","departure_time"]].apply(
    lambda x : compute_duration(
        x["arrival_time"], x["departure_time"]), axis = 1)

timetable["stop_name"] = timetable["stop_name"].swifter.progress_bar(True).apply(lambda x: fix_names(x))
len(UNKNOWNS)

Pandas Apply: 100%|██████████| 8108/8108 [03:32<00:00, 38.12it/s]  


1061

In [9]:
def remove_unknown_stops(row):
    to_remove = []
    for i, stop in enumerate(row.stop_name):
        if not stop:
            to_remove.append(i)
    if len(row.arrival_time) != len(row.stop_name):
        print('Whoa!')
        print(row)
        return row
    for i in sorted(to_remove, reverse=True):
        del row.arrival_time[i]
        del row.departure_time[i]
        # del row.stop_id[i]
        del row.stop_name[i]
        del row.stop_lat[i]
        del row.stop_lon[i]
        if i > 0:
            tmp = row.durations[i - 1]
            del row.durations[i - 1]
            if i < len(row.durations):
                row.durations[i - 1] += tmp
    return row

timetable.apply(remove_unknown_stops, axis=1)
timetable.drop(timetable[timetable.stop_name.str.len() <= 1].index, inplace=True)

In [10]:
timetable.head(3)

,trip_id,arrival_time,departure_time,stop_id,stop_name,stop_lat,stop_lon,durations
0,OCESN001001F0100459990,"[08:21:00, 11:06:00, 11:26:00, 11:52:00, 12:17...","[08:21:00, 11:09:00, 11:29:00, 12:00:00, 12:20...","[StopPoint:OCECorail Intercité-87113001, StopP...","[Paris-Est, Nancy-Ville, Lunéville, Sarrebourg...","[48.87656977, 48.68978225, 48.58799369, 48.737...","[2.35915061, 6.17427169, 6.49703457, 7.0527805...","[165.0, 17.0, 23.0, 17.0]"
1,OCESN001001F0100659999,"[07:06:00, 10:26:00, 10:50:00, 11:14:00, 11:31...","[07:06:00, 10:32:00, 10:52:00, 11:16:00, 11:33...","[StopPoint:OCECorail Intercité-87113001, StopP...","[Paris-Est, Nancy-Ville, Lunéville, Sarrebourg...","[48.87656977, 48.68978225, 48.58799369, 48.737...","[2.35915061, 6.17427169, 6.49703457, 7.0527805...","[200.0, 18.0, 22.0, 15.0]"
2,OCESN001001F0100759984,"[08:21:00, 11:06:00, 11:26:00, 11:52:00, 12:17...","[08:21:00, 11:09:00, 11:29:00, 12:00:00, 12:20...","[StopPoint:OCECorail Intercité-87113001, StopP...","[Paris-Est, Nancy-Ville, Lunéville, Sarrebourg...","[48.87656977, 48.68978225, 48.58799369, 48.737...","[2.35915061, 6.17427169, 6.49703457, 7.0527805...","[165.0, 17.0, 23.0, 17.0]"


In [11]:
timetable.to_csv("./data/export-intercites-gtfs-last/timetable_intercity.csv")

In [12]:
merged_ter = pd.merge(stop_times_ter,stops_ter,how='left', left_on=stop_times_ter.stop_id, right_on = stops_ter.stop_id) \
            [['trip_id','arrival_time','departure_time','stop_name','stop_lat','stop_lon']]

timetable_ter = merged_ter.groupby("trip_id").agg(list).reset_index()

timetable_ter["durations"] = timetable_ter[["arrival_time","departure_time"]].apply(
    lambda x : compute_duration(
        x["arrival_time"], x["departure_time"]), axis = 1)

timetable_ter["stop_name"] = timetable_ter["stop_name"].swifter.progress_bar(True).apply(lambda x: fix_names(x))

timetable_ter.apply(remove_unknown_stops, axis=1)
timetable_ter.drop(timetable_ter[timetable_ter.stop_name.str.len() <= 1].index, inplace=True)

Pandas Apply: 100%|██████████| 31984/31984 [00:54<00:00, 584.29it/s]


In [13]:
timetable_ter.head(3)

,trip_id,arrival_time,departure_time,stop_name,stop_lat,stop_lon,durations
223,OCESN002100R0100260086,"[06:25:00, 06:33:00, 06:40:00, 06:48:00]","[06:25:00, 06:34:00, 06:41:00, 06:48:00]","[Provins, Champbenoist-Poigny, Ste-Colombe-Sep...","[48.55569426, 48.54535739999999, 48.53017483, ...","[3.30284529, 3.28705609, 3.25721747, 3.24968453]","[8.0, 6.0, 7.0]"
224,OCESN002100R0100360088,"[06:30:00, 06:33:00, 06:40:00, 06:48:00]","[06:30:00, 06:34:00, 06:41:00, 06:48:00]","[Provins, Champbenoist-Poigny, Ste-Colombe-Sep...","[48.55569426, 48.54535739999999, 48.53017483, ...","[3.30284529, 3.28705609, 3.25721747, 3.24968453]","[3.0, 6.0, 7.0]"
225,OCESN002101R0100160094,"[06:44:00, 06:51:00, 06:57:00, 07:02:00]","[06:44:00, 06:52:00, 06:58:00, 07:02:00]","[Longueville, Ste-Colombe-Septveilles, Champbe...","[48.51351115, 48.53017483, 48.54535739999999, ...","[3.24968453, 3.25721747, 3.28705609, 3.30284529]","[7.0, 5.0, 4.0]"


In [14]:
timetable_ter.to_csv("./data/ter/timetable_ter.csv")